# 📊 Mô Tả Tập Dữ Liệu (Dataset Description)

## 1. Tổng quan dữ liệu
Tập dữ liệu bao gồm kết quả khảo sát từ **759 người đang đi làm**, tập trung vào các yếu tố ảnh hưởng đến nghề nghiệp và mức độ hài lòng.

| Đặc điểm | Chi tiết |
| :--- | :--- |
| **Số lượng quan sát** | 759 |
| **Số lượng biến** | 241  |
| **Định dạng** | Bảng (Tabular), dữ liệu số |
| **Giá trị khuyết (Null)** | Không phát hiện |
| **Biến mục tiêu** | `p1q9_job_current` (6 nhóm nghề) |

---

## 2. Phân nhóm biến
Các biến trong tập dữ liệu được chia thành các nhóm chính sau:

* **Thông tin nền (Demographics):** Tuổi, loại hình trường đào tạo, dân tộc và các đặc điểm cá nhân khác.
* **Hoạt động hướng nghiệp:** Các hoạt động diễn ra trong thời gian đi học.
* **Yếu tố ảnh hưởng nghề nghiệp:** Đo lường bằng thang đo Likert.
* **Mức độ hài lòng nghề nghiệp:**
    * Gồm 15 câu hỏi thành phần: `q12_rate1` đến `q12_rate15`.
    * **Biến tổng hợp:** `tong12` (Giá trị trung bình của 15 câu hỏi trên).

---

## 3. 🎯 Biến mục tiêu (Target Variable)
* **Tên biến:** `p1q9_job_current`
* **Mô tả:** Biến phân loại gồm 6 nhóm nghề (mã hóa từ `0` đến `5`).
* **⚠️ Lưu ý về phân phối lớp:** Dữ liệu bị **mất cân bằng nghiêm trọng (Imbalanced Data)**.
    * **Nhóm 0:** Chiếm ưu thế (> 50% mẫu).
    * **Nhóm 1–3:** Số lượng trung bình.
    * **Nhóm 4–5:** Rất ít quan sát (đặc biệt Nhóm 5 chiếm < 1%).

---

## 4. Đặc điểm xử lý dữ liệu

### ⚠️ Mã hóa đặc biệt (Special Encoding)
Trong tập dữ liệu xuất hiện các giá trị âm mang ý nghĩa định danh, **không có ý nghĩa số học**:
* `-1`: Không áp dụng.
* `-2`: Không trả lời.
> **Lưu ý:** Phải xử lý các giá trị này trước khi đưa vào mô hình tính toán.

### ℹ️ Thang đo hài lòng (Satisfaction Scale)
* 15 biến (`q12_rate1` – `q12_rate15`) tạo thành một thang đo nhất quán (Cronbach's alpha cao).
* Biến `tong12`: Phản ánh kết quả **sau khi đã đi làm**.
> **Quan trọng:** Biến `tong12` là kết quả (outcome), **không được sử dụng làm biến nguyên nhân (feature)** để dự báo nghề nghiệp hiện tại để tránh rò rỉ dữ liệu (data leakage).

# MỤC ĐÍCH CỦA TÔI LÀ HƯỚNG NGHIỆP CHO HỌC SINH DỰA VÀO THÔNG TIN KHI Ở TRƯỜNG HỌC 

# import thư viện

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import pandas as pd



# 🧹 Tiền Xử Lý: Loại Bỏ Biến (Feature Selection)

Để đảm bảo tính chính xác của mô hình và tránh hiện tượng **Rò rỉ dữ liệu (Data Leakage)**, các biến sau đây sẽ được loại bỏ khỏi tập dữ liệu huấn luyện.

Những biến này phản ánh trạng thái **sau khi đã đi làm** hoặc là **cảm nhận hiện tại**, do đó không thể dùng làm đầu vào để dự báo (vốn dĩ phải là dữ liệu có trước khi sự kiện xảy ra).

| Nhóm thông tin | Câu hỏi / Mô tả biến | Lý do loại bỏ |
| :--- | :--- | :--- |
| **Đánh giá hiện tại** | *"Tích vào những câu trả lời sau đây theo từng mức độ mà Anh/chị cho là đúng với mình"* | Phản ánh tâm lý hiện tại, không phải đặc điểm cố định. |
| **Cảm xúc hiện tại** | *"Hiện tại, bạn cảm thấy thế nào về lựa chọn của mình?"* | Dữ liệu thu thập sau khi sự việc đã xảy ra (Post-hoc evaluation). |
| **Lý do chọn trường** | *"Hãy cho biết vì sao Anh/chị quyết định chọn trường mà anh/chị đã thi..."* | Dữ liệu dạng văn bản mở (Text), khó định lượng trực tiếp và mang tính hồi tưởng. |
| **Liên hệ công việc** | *"Công việc hiện nay của anh/chị có liên quan gì đến nghề được học ở THCS không?..."* | **Data Leakage:** Tiết lộ trực tiếp thông tin về công việc hiện tại. |
| **Đánh giá chủ quan** | *"Nghề nghiệp mà anh/chị cho là thành công nhất nhờ việc hướng nghiệp ở THCS?"* | Mang tính chủ quan cá nhân về quá khứ. |
| **Lịch sử thay đổi** | *"Anh/chị đã từng thay đổi công việc?"* | Thông tin xảy ra trong quá trình đi làm (Future information). |
| **Kết quả đầu ra** | *"Nghề nghiệp của anh/chị sau khi học xong..."* | Đây là kết quả (Outcome), tương đương với biến mục tiêu. |
| **Biến tổng hợp** | **Biến `tong12`**: *"Trung bình những câu trả lời sau đây..."* | Biến tổng hợp từ thang đo hài lòng sau khi đi làm (Outcome variable). |

---
> **Kết luận:** Chỉ giữ lại các biến thuộc về **Thông tin nền (Demographics)** và **Yếu tố ảnh hưởng/Hướng nghiệp trong quá trình đi học** để xây dựng mô hình dự báo.

In [15]:

# ======================
# CONFIG
# ======================
DATA_PATH = "Dataset_Phieu_final.csv"
TARGET = "p1q9_job_current"


# ======================
# Helper
# ======================
def select_by_prefix(df, prefixes):
    return [c for c in df.columns if any(c.startswith(p) for p in prefixes)]


def select_existing(df, names):
    return [c for c in names if c in df.columns]


# ======================
# MAIN
# ======================
df = pd.read_csv(DATA_PATH)

if TARGET not in df.columns:
    raise ValueError(f"Không tìm thấy cột target '{TARGET}'")

# ---------- KEEP: THCS-only ----------
keep_exact = [
    # Nhân khẩu học / nền tảng
    "p1q5_gender",
    "v9",
    "p1q6_ethn",
    "p1q2_typeschool",
    "p1q3_typeclass",

    # Gia đình
    "p1q12_fjob",
    "p1q13_mjob",
    "p1q14_megr9",

    # Môn học
    "p1q15_hsub",
    "p1q16_lsub",
    "p1q16_likesub",
    "p1q16_dislikesub",

    # Hướng nghiệp
    "q1_trainjob",
    "q1a_namtrainjob",
    "q1c_cer",
    "q8_ins",
    "q9_insexam",
    "p1q8_afloseschool",
]

keep_prefixes = [
    "q2_care",
    "q3_where",
    "q4_act",
    "q5_affect",
    "q6_like",
    "requi",
    "aware",
]

keep_cols = []
keep_cols += select_existing(df, keep_exact)
keep_cols += select_by_prefix(df, keep_prefixes)

# bỏ trùng
keep_cols = list(dict.fromkeys(keep_cols))


# ---------- DROP: leakage / không THCS-only ----------
drop_exact = [
    "p1q8a_job_afloseschool",
    "p1q10_chanjob",
    "tong12",
]

drop_prefixes = [
    "q12_rate",
    "q10_whycho",
]

keep_cols = [
    c for c in keep_cols
    if (c not in drop_exact)
    and (not any(c.startswith(p) for p in drop_prefixes))
]

# ---------- Build filtered dataset ----------
df_filtered = df[keep_cols + [TARGET]].copy()

# ---------- Save ----------
df_filtered.to_csv("dataset_THCS_only_filtered.csv", index=False)

# ---------- Quick check ----------
print("Rows:", df_filtered.shape[0])
print("Features kept:", len(keep_cols))
print("Target distribution:")
print(df_filtered[TARGET].value_counts())


Rows: 759
Features kept: 184
Target distribution:
p1q9_job_current
0    398
1    141
3    111
2     92
4     14
5      3
Name: count, dtype: int64
